<a href="https://colab.research.google.com/github/Dimonfordont/DS_course/blob/main/Neural_networks/CV/Segementation/Breast_cancer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breast cancer segmentation

## Installs & Imports

In [ ]:
%%capture
#pip install catalyst

In [ ]:
#import albumentations as albs
import cv2
import io
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

from glob import glob
from sklearn.model_selection import train_test_split
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from IPython import display

#from catalyst import dl, utils

## Download dataset

In [ ]:
!curl -JLO 'https://www.dropbox.com/scl/fi/gs3kzp6b8k6faf667m5tt/breast-cancer-cells-segmentation.zip?rlkey=md3mzikpwrvnaluxnhms7r4zn'
!unzip breast-cancer-cells-segmentation.zip

## EDA

In [ ]:
!unzip breast-cancer-cells-segmentation.zip

Archive:  breast-cancer-cells-segmentation.zip
replace Images/ytma10_010704_benign1_ccd.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
imgs = glob(os.path.join("/content/Images", "*.tif"))

In [ ]:
print(os.getcwd())

In [ ]:
masks = [os.path.join("/content/Masks", i.rsplit("/",1)[-1].split("_ccd")[0]+".TIF") for i in imgs]

In [ ]:
print(masks)

In [ ]:
dataset_tuples = list(zip(imgs, masks))
random.shuffle(dataset_tuples)
train_tuples, test_tuples = train_test_split(dataset_tuples)

## Write Dataloaders and transform data

In [ ]:
SIZE = 512

In [ ]:
class BreastDataset(Dataset):

    def __init__(self, img_mask):
        self.img_mask = img_mask

    def __len__(self,):
        return len(self.img_mask)

    def __getitem__(self, idx):
        img_path, mask_path = self.img_mask[idx]
        image = self.get_transform(img_path)
        mask = self.get_transform(mask_path, normalize=False)
        mask[mask > 0] = 1
        return image,mask

    def transform_image(self, path, normalize=True,resize=(SIZE, SIZE)):
       image = io.imread(path)
       image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
       image = cv2.resize(image, resize)
       if normalize:
          return image/255
       return image

In [ ]:
train_dataset = BreastDataset(train_tuples)
test_dataset = BreastDataset(test_tuples)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## Get unet model

#### Create double convolution block

In [ ]:
class DoubleConv(nn.Module)
    def __init__(self, inch, ouch, kernel=3, padding=1):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(inch, ouch, kernel, padding=padding),
            nn.BatchNorm2d(ouch),
            nn.ReLU(inplace=True),
            nn.Conv2d(ouch, ouch, kernel, padding=padding),
            nn.BatchNorm2d(ouch),
            nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.conv(x)
        return x

#### Encoder part

In [ ]:
class Encoder(nn.Module):
  def __init__(self, inch, ouch):
    super().__init__()
    self.enconv = nn.Sequential(nn.MaxPool2d(2),
                                DoubleConv(inch, ouch))

    def forward(self,x):
      x = self.enconv(x)
      return x

SyntaxError: invalid syntax (<ipython-input-1-198ea377f0ee>, line 2)

#### Decoder part

In [ ]:
class Decoder(nn.Module):
  def __init__(self, inch, ouch):
    super().__init__()
    self.upscaler = nn.ConvTranspose2d(inch, ouch//2, kernel_size=2, stride=2)
    self.conv = DoubleConv(inch, ouch)


    def forward(self, x1, x2):
      x1 = self.upscaler(x1)
      x1s = x1.size()
      x2s = x2.size()
      Xdiff = x2s[3] - x1s[3]
      Ydiff = x2s[2] - x1s[2]
      mdiffX = Xdiff - Xdiff//2
      mdiffY = Ydiff - Ydiff//2
      x1 = F.pad(x1, [Xdiff//2, mdiffX, Ydiff//2, mdiffY])
      x = torch.cat([x2, x1], dim=1)
      x = self.conv(x)
      return x

NameError: name 'nn' is not defined

#### Basic output convolution layer

In [ ]:
class ConvOut(nn.Module):
  def __init__(self, inch, ouch):
    super(ConvOut, self).__init__()
    self.conv = nn.Conv2d(inch, ouch, kernel_size=1)


  def forward(self, x):
    return self.conv(x)

#### combine it all together

In [ ]:
class Unet(nn.Module):
  def __init__(self, n_in, n_out):
    super(Unet, self).__init__()
    self.n_channels = n_in
    self.n_classes = n_out
    d64, d128, d256, d512, d1k = 64, 128, 256, 512, 1024

    self.inc = DoubleConv(n_in, d64)
    self.enc1 = Encoder(d64, d128)
    self.enc2 = Encoder(d128, d256)
    self.enc3 = Encoder(d256, d512)
    self.enc4 = Encoder(d512, d1k)

    self.dec4 = Decoder(d1k, d512)
    self.dec3 = Decoder(d512, d256)
    self.dec2 = Decoder(d256, d128)
    self.dec1 = Decoder(d128, d64)

    self.out = ConvOut(d64, n_out)


  def forward(self, x):
    x1 = self.inc(x)
    x2 = self.enc1(x1)
    x3 = self.enc2(x2)
    x4 = self.enc3(x3)
    x = self.enc4(x4)

    x = self.dec4(x, x4)
    x = self.dec3(x, x3)
    x = self.dec2(x, x2)
    x = self.dec1(x, x1)

    x = self.out(x)
    return x

### Set a config

In [ ]:
class CFG:
  epochs = 10
  lr = 4e-4
  scheduler = False
  wandb = False
  device = lambda: "cuda" if torch.cuda.is_available() else "cpu"
  input_size = 224

In [ ]:
def get_setup(model=False, optimizer=False, loss=False):
  model = model or Unet(n_in=3, n_out=1)
  model.to(CFG.device())
  optimizer = optimizer or torch.optim.Adam(model.parameters(), lr=CFG.lr)
  criterion = loss or nn.BCEWithLogitsLoss()
  return, model, optimizer, criterion

In [ ]:
model, optimizer, criterion = get_setup()

### train loop

In [ ]:
def train(model, optimizer, criterion, traindata):
  mpdel.train()
  for epoch in tqdm(CFG.epochs):
    for imgs, lbls in iter(traindata):
      imgs = imgs.to(CFG.device()).permute(0, 3, 2, 1).float()
      lbls = lbls.to(CFG.device()).permute(0, 3, 2, 1).float()
      lbls = lbls.sum(1, keepdim=True).bool().float()
      with optimizer.zero_grad():

        logits = model(imgs)
        loss = criterion(logits, lbls)
        loss.backward()
        optimizer.step()
  return model